<a href="https://colab.research.google.com/github/SoufianeNoubir/Soufiane-Noubir-EEG-EPQ-2020-2021/blob/main/Spatial_Imagery_Commented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing library for GDF files

In [ ]:
pip install mne


     |████████████████████████████████| 6.9MB 5.9MB/s 


Mounting Google drive to colab

In [ ]:
#Mounting Google drive to colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Loading in the relevant files

In [ ]:
cd /content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


/content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


In [ ]:
!pwd

/content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


Importing the libraries

In [ ]:
#Importing the libraries
import os
import numpy as np
import mne
from scipy.linalg import eigh
import math
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


Loading the dataset from the files

In [ ]:
#Loading the dataset from the files
#Due to time limitations described in the report I will only be using one participant whose data is in the file "A01T.gdf"
#This participant used an EEG with a sample rate of 200Hz.
raw=mne.io.read_raw_gdf('A01T.gdf')
#here we are extracting the actual EGG data from raw
data=raw.get_data()
#here we are extracting the labels from the dataset. 
#There is a label indicating when there is a change of state i.e. when the participant stops doing a certain action and starts another
#Each label is a number which indicates the activity the participant has just started. I am only interested in the labels for imagining the tongue,feet,left and right arms.
events_A, event_id_A = mne.events_from_annotations(raw)

Extracting EDF parameters from /content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-7-b279547f86f7>:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf('A01T.gdf')


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


This code snippet finds the the start points of all the relevant labels and saves the next 3 seconds(according to the documentation of this dataset each trial is 3 seconds) worth of data in the relevant dataset. Some of the data is invalid and these are marked by the reject label.


In [ ]:
#splitting the dataset up by label.
#This code snippet finds the the start points of all the relevant labels.
#Some of the data is invalid and these are marked by the reject label.
#The position of the relevant labels is scaled down by a factor of 2 as later the dataset will be downsampled to 125Hz.
#The imagination of the tongue represents moving the cursor up.
#The imagination of the feet represents moving the cursor down.
#The imagination of the right arm represents moving the cursor to the right.
#The imagination of the left arm represents moving the cursor to the left.
rejections=[]
left=[]
right=[]
up=[]
down=[]

for i in events_A:
  if i[2]==1:
    if i[0]%2==0:
      rejections.append((i[0])/2)
    else:
      rejections.append(((i[0])+1)/2)
  if i[2]==7:
    if i[0]%2==0:
      left.append((i[0])/2)
    else:
      left.append(((i[0])+1)/2)
  if i[2]==8:
    if i[0]%2==0:
      right.append((i[0])/2)
    else:
      right.append(((i[0])+1)/2)
  if i[2]==10:
    if i[0]%2==0:
      up.append((i[0])/2)
    else:
      up.append(((i[0])+1)/2)
  if i[2]==9:
    if i[0]%2==0:
      down.append((i[0])/2)
    else:
      down.append(((i[0])+1)/2)

Function that removes unwanted electrodes

In [ ]:
#removes unwanted electrodes
#As discussed in report we only want the first 22 electrodes and thus the last 3 are omitted from our data.
def remove_electrodes(data,numbers):
  output=[]
  for i in range(len(data)):
    if i in numbers:
      
      output.append(data[i])
  
  return output
data=remove_electrodes(data,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

Function that downsamples by  a factor of 2 from 250Hz to 125Hz by only using using every second value.

In [ ]:
#downsamples data by a given factor
def downsample(data,factor):
  output=[]
  current=[]
  for i in data:
    for j in range(len(i)):
      if j%factor==0:
        current.append(i[j])
    output.append(current)
    current=[]
  return output
#here we are downsampling by a factor of 2
data_downsample=downsample(data,2)


Splitting up the data into 4 classes

In [ ]:
#splits data up into segments of EEG data corresponding to one of the 4 classes
#this is done by using the positions of each of the trials and extracting the relevant 3 seconds of EEG data if the trial is not a reject.

#up_data contains the downsampled data for moving the cursor up.
up_data=[]
#down_data contains the downsampled data for moving the cursor down.
down_data=[]
#left_data contains the downsampled data for moving the cursor left.
left_data=[]
#right_data contains the downsampled data for moving the cursor right.
right_data=[]
current=[]
for i in up:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    up_data.append(current)
    current=[]
for i in down:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    down_data.append(current)
    current=[]
for i in right:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    right_data.append(current)
    current=[]
for i in left:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    left_data.append(current)
    current=[]

Splitting the data into training and testing

In [ ]:
#splitting of dataset into training and testing data

#left_data_evaluation contains the test data for moving the cursor left and so on
#datapoints from left_data is removed and placed into left_data_evaluation with a probability of 1/4. 
#Therefore, overall the models will be trained on 3/4 the size of initial dataset and tested on the other quarter.
#same is done for all 4 classes
left_data_evaluation=[]
right_data_evaluation=[]
down_data_evaluation=[]
up_data_evaluation=[]
import random
for i in up_data:
  x=random.randint(1,4)
  if x==4:
    up_data.remove(i)
    up_data_evaluation.append(i)
for i in right_data:
  x=random.randint(1,4)
  if x==4:
    right_data.remove(i)
    right_data_evaluation.append(i)
for i in down_data:
  x=random.randint(1,4)
  if x==4:
    down_data.remove(i)
    down_data_evaluation.append(i)
for i in left_data:
  x=random.randint(1,4)
  if x==4:
    left_data.remove(i)
    left_data_evaluation.append(i)

General data manipulation. Nothing very important or relevant is done here.

In [ ]:

#space_data is simply a collection of all the training data
#It will be useful as it contains all the trials but are seperated by which class they belong to
space_data=[]
space_data.append(up_data)
space_data.append(right_data)
space_data.append(down_data)
space_data.append(left_data)
#space_data_evaluation is the exact same thing except for the training data
space_data_evaluation=[]
space_data_evaluation.append(up_data_evaluation)
space_data_evaluation.append(right_data_evaluation)
space_data_evaluation.append(down_data_evaluation)
space_data_evaluation.append(left_data_evaluation)
#x_all_test and y_all_test serve a similar purpose but have a slightly different
#y_all_test contains the labels for the test data 
#class 1 is for up
#class 2 is for right
#class 3 is for down
#class 4 is for left 
#we will be using this labelling of the 4 classes throughout the rest of the code.
x_all_test=[]
y_all_test=[]
x_all_test+=up_data_evaluation
x_all_test+=right_data_evaluation
x_all_test+=down_data_evaluation
x_all_test+=left_data_evaluation
for i in up_data_evaluation:
  y_all_test.append(1)
for i in right_data_evaluation:
  y_all_test.append(2)
for i in down_data_evaluation:
  y_all_test.append(3)
for i in left_data_evaluation:
  y_all_test.append(4)

Implementation of standard CSP. CSP is an algorithm that as an input takes data that belongs to 2 different classes and finds filters that when the data is projected onto leaves a high variance in only one class compared to the other. This needs to be done for both classes i.e.(both classes need filters that will maximise the variance in that class compared to the other). This allows us to extract features that will help distinguish the 2 different classes. See report for more details.

In [ ]:
#implementation of standard CSP approach

#data_1 and data_2 are datasets for 2 different classes on which we are trying to find the optimal set of filters that will distinguish between them

#standard CSP can only take 2 different classes
def csp(data_1,data_2):
  #mean_cov_1 is the mean class covariance matrix for the first class
  mean_cov_1=np.cov(np.array(data_1[0]))
  #mean_cov_2 is the mean class covariance matrix for the second class
  mean_cov_2=np.cov(np.array(data_2[0]))
 
  for i in data_1[1:]:
    #iterating through the first dataset and adding on the covariance matrix for each data point
    mean_cov_1+=np.cov(np.array(i))
    
  for i in data_2[1:]:
    #iterating through the second dataset and adding on the covariance matrix for each datapoint
    mean_cov_2+=np.cov(np.array(i))
  #here we divide by the number of datapoints in each class therefore producing our 2 mean class covariance matrices 
  mean_cov_1=mean_cov_1/len(data_1)
  mean_cov_2=mean_cov_2/len(data_2)

  #here we are generating the eigenvalues and eigenvectors of the matrix discussed in the appendix 
  #technically the matrix used here is different but the eigendecomposition of this matrix is mathematically equivalent to the one in the report and thus there is absolutely no difference
  #the eigenvectors of this matrix is exactly the same as the eigenvectors of the matrix given in the report
  #the eigenvalues will be different but the ordering of these eigenvalues will be the same
  #i.e if you were to order the eigenvectors of this matrix in descending order of eigenvalue they would be in the same order as if you did the same with the matrix in the report
  #thus the filters outputted will be the same
  #the exact mathematical reasoning for this is not relevant
  w_1, v_1 = np.linalg.eig(np.dot(np.linalg.inv(mean_cov_2+mean_cov_1),mean_cov_1))
  sort=np.argsort(w_1)
  #returning the eigenvectors correponding to the 3 smalles and largest eigenvalues
  #these eigenvectors represent directions in space that when projected down onto leave a very high variance in one of the 2 classes and a very low variance in the other
  #this allows us to distinguish between the 2 classes
  #the difference between the vectors with the smallest and largest eigenvalues is which class the variance is being maximised in.
  return ([v_1[sort[0]],v_1[sort[1]],v_1[sort[2]],v_1[sort[-3]],v_1[sort[-2]],v_1[sort[-1]]])

Implementation of multiclass CSP. This is a generalisation of standard CSP that works on multiclass data. It is inspired by LDA. It finds filters that when the data is projected onto maximises the between class scatter to the within class scatter. This means that the spread between classes is large but each class forms its own small cluster making them easy to distinguish.

In [ ]:
#implementation of multiclass CSP

# here data is a collection of data that belong to more than 2 classes i.e multiclass data.
# as discussed in the report my generalisation of standard CSP is based on the mean scatter matrices rather than the mean covariance matrices.

def multiclass_csp(data):
  #this section of code is simply using the formula shown in the report to find the mean class scatter matrices

  #total_elements finds the total number of datapoints in all classes
  total_elements=0
  for i in data:
    total_elements+=len(i)
  #average is a list that contains the class average for all classes
  average=[]
  for i in data:
    total=np.array(i[0])
    
    for j in i[1:]:
      
      total+=np.array(j)
    total=total/len(i)
    
      
    average.append(total)
  total_average=len(data[0])*average[0]
  for i in range(len(data[1:])):
    total_average+=len(data[i])*average[i]
  total_average=total_average/total_elements
  within_class=np.dot(data[0][0]-average[0],(data[0][0]-average[0]).transpose())
  for i in range(len(data)):
    
    for j in data[i]:
      within_class+=np.dot(j-average[i],(j-average[i]).transpose())
  within_class-=np.dot(data[0][0]-average[0],(data[0][0]-average[0]).transpose())
  #this scaling of the within_class scatter matrix was simply to make the numbers more manageable during debugging and has no mathematical significance. 
  within_class=within_class/total_elements
  between_class=len(data[0])*(np.dot(average[0]-total_average,(average[0]-total_average).transpose()))
  for i in range(len(average[1:])):
    between_class+=len(data[i])*(np.dot(average[i]-total_average,(average[i]-total_average).transpose()))
  #this scaling of the between_class scatter matrix was simply to make the numbers more manageable during debugging and has no mathematical significance.
  between_class=between_class/len(data)


  #here we are generating the eigenvalues and eigenvectors of the matrix discussed in the appendix section for LDA.
  #technically the matrix used here is different but the eigendecomposition of this matrix is mathematically equivalent to the one in the report and thus there is absolutely no difference
  #the eigenvectors of this matrix is exactly the same as the eigenvectors of the matrix given in the report
  #the eigenvalues will be different but the ordering of these eigenvalues will be the same
  #i.e if you were to order the eigenvectors of this matrix in descending order of eigenvalue they would be in the same order as if you did the same with the matrix in the report
  #thus the filters outputted will be the same
  #the exact mathematical reasoning behind why both the eigendecompostions are exactly the same is not relevant
  #the reason I am using this slightly adapted matrix is because it is easier to decompose
  w_1, v_1 = np.linalg.eig(np.dot(np.linalg.inv(between_class+within_class),between_class))
  sort=np.argsort(w_1)
  #here since we are trying to maximise the ratio of the between class scatter to the within class scatter we do not consider the eigenvectors with the smallest eigenvalues
  return ([v_1[sort[0]],v_1[sort[1]],v_1[sort[2]],v_1[sort[3]],v_1[sort[4]],v_1[sort[5]]])



Feature extraction. This algorithm takes the filters produced by CSP and uses them to extract feautres that represent the data which a classifier can classify. 

In [ ]:
#implementation of feature extraction using filters generated from CSP
#here the input is up_data which is simply a matrix that represents a single trial of EEG data
#the filters are all in turn being applied to this matrix resulting in a vector each time
#vectors are arrays of number which we simply take the variance of
#these variances are the features that will be used for classification
def feature_extraction(up_data,filters):
  features=[]
  for i in filters:
    #np.dot() function applies the filter to the data
    #np.var() function takes the variance of the resulting vector
    #np
    features.append(math.log(np.var(np.dot(i.transpose(),up_data))))
    
  return features

Generating the filters for all the CSP based classification we will be doing.

In [ ]:
#the 3 different CSP classification approaches are describes in detail in the report

#one of them is based off OvM classification
#this consists of training 4 binary classifiers to distinguish between up and the rest, down and the rest, right and rest, left and rest
#for example csp_filters_left are filters that will distinguish between left versus up, down and right
csp_filters_left=csp(left_data,right_data+up_data+down_data)
csp_filters_right=csp(right_data,left_data+up_data+down_data)
csp_filters_down=csp(down_data,right_data+up_data+left_data)
csp_filters_up=csp(up_data,right_data+left_data+down_data)

#the other was based on OvO classification
#this is based on training 6 binary classifiers to distinguish between every pair of classes i.e up v down, right v up etc..
#for example csp_filters_up_down are filters that will distinguish between up and down
csp_filters_up_down=csp(up_data,down_data)
csp_filters_down_right=csp(right_data,down_data)
csp_filters_up_right=csp(up_data,right_data)
csp_filters_down_left=csp(left_data,down_data)
csp_filters_right_left=csp(right_data,left_data)
csp_filters_up_left=csp(up_data,left_data)
#the final csp based approach was binary search which first requires distinguishing between vertical and horizontal directions
#csp_filters_v_h will distinguish between the up/down direction and the right/left direction
#once this classification is done then we do a up v down classification or a right v left classification.
csp_filters_v_h=csp(up_data+down_data,right_data+left_data)
#csp_multiclass are filters that are generated using my multiclass generalisation of CSP that will distinguish between all 4 classes 
csp_multiclass=multiclass_csp(space_data)

This section of code is initialising all the datasets for all the classifications

In [ ]:
x_vertical_horizontal_train=[]
y_vertical_horizontal_train=[]
x_vertical_horizontal_test=[]
y_vertical_horizontal_test=[]
x_up_down_train=[]
y_up_down_train=[]
x_up_down_test=[]
y_up_down_test=[]
x_left_right_train=[]
y_left_right_train=[]
x_left_right_test=[]
y_left_right_test=[]
x_left_train=[]
y_left_train=[]
x_left_test=[]
y_left_test=[]
x_right_train=[]
y_right_train=[]
x_right_test=[]
y_right_test=[]
x_up_train=[]
y_up_train=[]
x_up_test=[]
y_up_test=[]
x_down_train=[]
y_down_train=[]
x_down_test=[]
y_down_test=[]
x_right_up_train=[]
y_right_up_train=[]
x_right_up_test=[]
y_right_up_test=[]
x_right_down_train=[]
y_right_down_train=[]
x_right_down_test=[]
y_right_down_test=[]
x_left_up_train=[]
y_left_up_train=[]
x_left_up_test=[]
y_left_up_test=[]
x_left_down_train=[]
y_left_down_train=[]
x_left_down_test=[]
y_left_down_test=[]
x_csp_space_train=[]
y_csp_space_train=[]
x_csp_space_test=[]
y_csp_space_test=[]
x_train_nn_space=[]
y_train_nn_space=[]
x_test_nn_space=[]
y_test_nn_space=[]

This section of code is actually creating the datasets. It is very repetitive so I will explain the code for only a few datasets and the rest is self-explanatory.

In [ ]:
#the x_vertical_horizontal_train is a dataset which contains the features for distinguishing between the vertical and horizontal directions that will be used for training
#the dataset is generated by iterating through the all trials in all the 4 classes 
#then the feature_extraction function is applied to each individual trial using csp_filters_v_h which are filters which distiniguish beetween horizontal and vertical directions
for i in up_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in down_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in right_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in left_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
#the exact same is done for x_vertical_horizontal_test which is the dataset that will be used for testing
for i in up_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in down_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in right_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in left_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
#the x_up_down_train is a dataset which contains the features for distinguishing between the up and down directions that will be used for training
#the dataset is generated by iterating through the all trials in all the up and down classes
#then the feature_extraction function is applied to each individual trial using csp_filters_up_down which are filters which distiniguish beetween up and down directions
for i in up_data:
  x_up_down_train.append(feature_extraction(i,csp_filters_up_down))
for i in down_data:
  x_up_down_train.append(feature_extraction(i,csp_filters_up_down))
#the exact same is done for x_up_down_test which is the dataset that will be used for testing
for i in up_data_evaluation:
  x_up_down_test.append(feature_extraction(i,csp_filters_up_down))
for i in down_data_evaluation:
  x_up_down_test.append(feature_extraction(i,csp_filters_up_down))

for i in right_data:
  x_left_right_train.append(feature_extraction(i,csp_filters_right_left))
for i in left_data:
  x_left_right_train.append(feature_extraction(i,csp_filters_right_left))

for i in right_data_evaluation:
  x_left_right_test.append(feature_extraction(i,csp_filters_right_left))
for i in left_data_evaluation:
  x_left_right_test.append(feature_extraction(i,csp_filters_right_left))

for i in right_data:
  x_right_up_train.append(feature_extraction(i,csp_filters_up_right))
for i in up_data:
  x_right_up_train.append(feature_extraction(i,csp_filters_up_right))

for i in right_data_evaluation:
  x_right_up_test.append(feature_extraction(i,csp_filters_up_right))
for i in up_data_evaluation:
  x_right_up_test.append(feature_extraction(i,csp_filters_up_right))

for i in right_data:
  x_right_down_train.append(feature_extraction(i,csp_filters_down_right))
for i in down_data:
  x_right_down_train.append(feature_extraction(i,csp_filters_down_right))

for i in right_data_evaluation:
  x_right_down_test.append(feature_extraction(i,csp_filters_down_right))
for i in down_data_evaluation:
  x_right_down_test.append(feature_extraction(i,csp_filters_down_right))

for i in left_data:
  x_left_up_train.append(feature_extraction(i,csp_filters_up_left))
for i in up_data:
  x_left_up_train.append(feature_extraction(i,csp_filters_up_left))

for i in left_data_evaluation:
  x_left_up_test.append(feature_extraction(i,csp_filters_up_left))
for i in up_data_evaluation:
  x_left_up_test.append(feature_extraction(i,csp_filters_up_left))


for i in left_data:
  x_left_down_train.append(feature_extraction(i,csp_filters_down_left))
for i in down_data:
  x_left_down_train.append(feature_extraction(i,csp_filters_down_left))

for i in left_data_evaluation:
  x_left_down_test.append(feature_extraction(i,csp_filters_down_left))
for i in down_data_evaluation:
  x_left_down_test.append(feature_extraction(i,csp_filters_down_left))


for i in up_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in down_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in right_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in left_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))

for i in up_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in down_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in right_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in left_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))

for i in up_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in down_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in right_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in left_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))

for i in up_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in down_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in right_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in left_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))

for i in up_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in down_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in right_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in left_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))

for i in up_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in down_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in right_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in left_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))

for i in up_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in down_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in right_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in left_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))

for i in up_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in down_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in right_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in left_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
#x_csp_space_train is the dataset that will be used for multiclass classification to distinguish between all 4 classes at once
#the dataset is generated by iterating through all trials in all 4 classes
#then the feature_extraction function is applied to each individual trial using csp_space which are filters which distiniguish beetween all directions
for i in up_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in down_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in right_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in left_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
#the exact same is done for x_csp_space_test which is the dataset that will be used for testing
for i in up_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in down_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in right_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in left_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))

#this section of code is for creating the dataset for the CNN
#x_train_nn_space is the dataset for training the CNN
#y_train_nn_space contains the correct labels for the x_train_nn_space
for i in space_data:
  
  for j in i:
    intermediate=np.transpose(j)
    intermediate_3=[]
    for k in intermediate:
      intermediate_2=[]
      for l in k:
        intermediate_2.append(np.array([l]))
      intermediate_3.append(np.array(intermediate_2))
    x_train_nn_space.append(np.array(intermediate_3))
    #to generate the data in x_train_space not much processing needs to be done
    #firstly the matrix representing each trial is transposed so that it fits the dimension of the CNN that will be trained
    #then since CNNs are usually applied to images with multiple channels(RGB channels) I need to turn each individual EEG reading into a list of length 1 to indicate there is only one channel
    #therefore each matrix will be of dimension (375,22,1) 375 for the number of recordings by each electrode, 22 for the number of electrodes and 1 to indicate only one channel per electrode.
    #The CNN will now be able to train on this 2 dimensional data without further processing
#since the CNN will be trained using categorical cross entropy loss the correct labels should be lists of length 4
#each element of the list gives the probability that the element is in each class
#These probabilities will all be 0 except for a probability of 1 in the correct class
#for example the correct label for a trial in the first class is [1,0,0,0]
for i in up_data:
  y_train_nn_space.append(np.array([1,0,0,0]))
for i in right_data:
  y_train_nn_space.append(np.array([0,1,0,0]))
for i in down_data:
  y_train_nn_space.append(np.array([0,0,1,0]))
for i in left_data:
  y_train_nn_space.append(np.array([0,0,0,1]))
# the same is done for the test data and the labels for the test data  
for i in space_data_evaluation:
  
  for j in i:
    intermediate=np.transpose(j)
    intermediate_3=[]
    for k in intermediate:
      intermediate_2=[]
      for l in k:
        intermediate_2.append(np.array([l]))
      intermediate_3.append(np.array(intermediate_2))   
    
    x_test_nn_space.append(np.array(intermediate_3))
    
for i in up_data_evaluation:
  y_test_nn_space.append(np.array([1,0,0,0]))
for i in right_data_evaluation:
  y_test_nn_space.append(np.array([0,1,0,0]))
for i in down_data_evaluation:
  y_test_nn_space.append(np.array([0,0,1,0]))
for i in left_data_evaluation:
  y_test_nn_space.append(np.array([0,0,0,1]))


Here the labels for the datasets for the CSP based approach are made. Like in the previous section the code is very repetitive and I will only explain a few.

In [ ]:
#y_vertical_horizontal_train contians the correct labels for x_horizontal_vertical_train
#the label 1 is given for any direction in the vertical direction
#the label 2 is given for any direction in the horizontal direction
for i in up_data:
 y_vertical_horizontal_train.append(1)
for i in down_data:
  y_vertical_horizontal_train.append(1)
for i in right_data:
  y_vertical_horizontal_train.append(2)
for i in left_data:
  y_vertical_horizontal_train.append(2)
#the same is done for y_vertical_horizontal_test which is the set of labels for the test data
for i in up_data_evaluation:
  y_vertical_horizontal_test.append(1)
for i in down_data_evaluation:
  y_vertical_horizontal_test.append(1)
for i in right_data_evaluation:
  y_vertical_horizontal_test.append(2)
for i in left_data_evaluation:
  y_vertical_horizontal_test.append(2)
#y_up_down_train contains the correct labels for x_up_down_train
#the labelling system descrive earlier is used
#the label 1 is given for the up direction
#the label 3 is given for the down direction
for i in up_data:
  y_up_down_train.append(1)
for i in down_data:
  y_up_down_train.append(3)
#the same is done for y_up_down_test which is the set of labels for the test data
for i in up_data_evaluation:
  y_up_down_test.append(1)
for i in down_data_evaluation:
  y_up_down_test.append(3)

for i in right_data:
  y_left_right_train.append(2)
for i in left_data:
  y_left_right_train.append(4)

for i in right_data_evaluation:
  y_left_right_test.append(2)
for i in left_data_evaluation:
  y_left_right_test.append(4)

for i in right_data:
  y_right_up_train.append(2)
for i in up_data:
  y_right_up_train.append(1)

for i in right_data_evaluation:
  y_right_up_test.append(2)
for i in up_data_evaluation:
  y_right_up_test.append(1)

for i in right_data:
  y_right_down_train.append(2)
for i in down_data:
  y_right_down_train.append(3)

for i in right_data_evaluation:
  y_right_down_test.append(2)
for i in down_data_evaluation:
  y_right_down_test.append(3)

for i in left_data:
  y_left_up_train.append(4)
for i in up_data:
  y_left_up_train.append(1)

for i in left_data_evaluation:
  y_left_up_test.append(4)
for i in up_data_evaluation:
  y_left_up_test.append(1)

for i in left_data:
  y_left_down_train.append(4)
for i in down_data:
  y_left_down_train.append(3)

for i in left_data_evaluation:
  y_left_down_test.append(4)
for i in down_data_evaluation:
  y_left_down_test.append(3)

for i in up_data:
  y_left_train.append(2)
for i in down_data:
  y_left_train.append(2)
for i in right_data:
  y_left_train.append(2)
for i in left_data:
  y_left_train.append(1)

for i in up_data_evaluation:
  y_left_test.append(2)
for i in down_data_evaluation:
  y_left_test.append(2)
for i in right_data_evaluation:
  y_left_test.append(2)
for i in left_data_evaluation:
  y_left_test.append(1)

for i in up_data:
  y_right_train.append(2)
for i in down_data:
  y_right_train.append(2)
for i in right_data:
  y_right_train.append(1)
for i in left_data:
  y_right_train.append(2)

for i in up_data_evaluation:
  y_right_test.append(2)
for i in down_data_evaluation:
  y_right_test.append(2)
for i in right_data_evaluation:
  y_right_test.append(1)
for i in left_data_evaluation:
  y_right_test.append(2)

for i in up_data:
  y_up_train.append(1)
for i in down_data:
  y_up_train.append(2)
for i in right_data:
  y_up_train.append(2)
for i in left_data:
  y_up_train.append(2)

for i in up_data_evaluation:
  y_up_test.append(1)
for i in down_data_evaluation:
  y_up_test.append(2)
for i in right_data_evaluation:
  y_up_test.append(2)
for i in left_data_evaluation:
  y_up_test.append(2)

for i in up_data:
  y_down_train.append(2)
for i in down_data:
  y_down_train.append(1)
for i in right_data:
  y_down_train.append(2)
for i in left_data:
  y_down_train.append(2)

for i in up_data_evaluation:
  y_down_test.append(2)
for i in down_data_evaluation:
  y_down_test.append(1)
for i in right_data_evaluation:
  y_down_test.append(2)
for i in left_data_evaluation:
  y_down_test.append(2)
#y_csp_space_train contians the correct labels for x_csp_space_train
#the label 1 is given for going up
#the label 2 is given for going right
#the label 3 is given for going down
#the label 4 is given for going left
for i in up_data:
  y_csp_space_train.append(1)
for i in down_data:
  y_csp_space_train.append(3)
for i in right_data:
  y_csp_space_train.append(2)
for i in left_data:
  y_csp_space_train.append(4)
#the same is done for y_csp_space_test which is the set of labels for the test data
for i in up_data_evaluation:
  y_csp_space_test.append(1)
for i in down_data_evaluation:
  y_csp_space_test.append(3)
for i in right_data_evaluation:
  y_csp_space_test.append(2)
for i in left_data_evaluation:
  y_csp_space_test.append(4)

Implementation of the OvO and OvM functions. These take the relevant outputs from classifiers and uses the OvO and OvM generalisation to make a classification.

In [ ]:
#uses one_v_one classification to output the predicted class
#takes the output of 6 classifiers which distiniguish any 2 classes i.e up v down classifier, down v right classifier, up v right classifier etc.

def one_v_one(outputs):
  #counts the number of classifiers that output class 1,2,3,4
  num_1=outputs.count(1)
  num_2=outputs.count(2)
  num_3=outputs.count(3)
  num_4=outputs.count(4)
  num=[num_1,num_2,num_3,num_4]
  #the class with the most votes is selected
  if num.count(max(num))==1:
    return (num.index(max(num)))+1
  
  #if there is a 2 way tie the output of the classifier used to distinguish between the 2 leading classes is used
  #for example if classes 1 and 2 got 2 votes and 3 and 4 each got one vote then the chosen class will be the output of the up v right classifier
  else:
    
    winners=[]
    for i in range(len(num)):
      if num[i] ==max(num):
        winners.append(i+1)
    #as discussed in the report the chance of a 3 way tie is unlikely and I will not be considering this case
    #however, if one does occur, the code will output an alert message to notify me
    if len(winners)==3:
      return ("alert")
    else:
      if min(winners)==1:
        return outputs[sum(winners)-3]
      else:
        return outputs[sum(winners)-2]
#uses one_v_many classification to output the predicted class
#takes the output of 4 classifiers which distinguish between one class and rest i.e. up v right,down,left, right v up,down,left, down v up,right,left, left v up,right,down 
def one_v_many(probs):
  #this functions takes in the confidence of each model in the individual class
  #the confidence in class k is the posterior probability i.e P(y=k,x)
  #the confidence tells us how likely it is that the input belongs to the one class
  #for example the first value in the list probs will be the confidence of the up v right,down,left model in the up class
  #the class whose model has the highest confidence in it is chosen
  return (probs.index(max(probs))+1)
#uses binary search to output the predicted class
def binary_search(outputs):
  #binary search works by distinguishing between vertical and horizontal directions
  if outputs[0]==1:
    #if the input is in the vetical direction then the output of the up v down classifier is used
    return outputs[1]
  else:
    #if the input is in the horizontal direction then the output of the right v left classifier is used
    return outputs[2]
#returns accuracy
def accuracy(outputs,correct):
  #this is my custom accuracy function that takes then outputs of my classification algorithm and checks it againsts the correct labels for the test data.
  right=0
  for i in range(len(outputs)):
    if outputs[i]==correct[i]:
      right+=1
  return right/(len(outputs))

Here I implement each of these approaches using LDA. OvO,OvM and bianry search appraoches are done using LDA. I use multiclass LDA for the features extracted by my multiclass CSP algorithm. LDA works by by fidning a projection that will maximise the ratio of the between class scatter to the within class scatter. The between class scatter gives a sense of how far apart differen classes are whereas within class scatter gives a sense of how spread out each individual class. If we maximise this ratio then it will become easier to distinguish between different classes. Classification using LDA is done by projecting onto this optimal filter and assigning the input to the class whose mean it is closest to. We can get the confidence in the each class by assuming each class is normally distributed with the same covariance matrix and then applying bayes formula. See appendix for details. Once again the code is very repetitive I will only comment a few sections and the rest is self explanatory.

In [ ]:
#this secction creates the muticlass LDA model that will classify features extracted using my multiclass CSP algorithm
multiclass_LDA = LinearDiscriminantAnalysis()
#it is being trained on x_csp_space_train with the labels y_csp_space_train
multiclass_LDA.fit(x_csp_space_train,y_csp_space_train)
#here I am evaluating the accuracy of the model on the test data c_csp_space_test with the labels y_csp_space_test
print(multiclass_LDA.score(x_csp_space_test,y_csp_space_test))

#this section creates the LDA model that will classify features extracted using standard CSP.
#the model will distinguish between the vertical and horizontal directions
LDA_binary_search_v_h = LinearDiscriminantAnalysis()
#it is being trained on x_vertical_horizontal_train with the labels y_vertical_horizontal_train
LDA_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
#here I am evaluating the accuracy of the model on the test data x_vertical_horizontal_test with the labels y_vertical_horizontal_test
print(LDA_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

#this section creates the LDA model that will classify features extracted using standard CSP.
#the model will distinguish between up and down
LDA_ovo_u_d = LinearDiscriminantAnalysis()
#it is being trained on x_up_down_train with the labels y_up_down_train
LDA_ovo_u_d.fit(x_up_down_train,y_up_down_train)
#here I am evaluating the accuracy of the model on the test data x_up_down_test with the labels y_up_down_test
print(LDA_ovo_u_d.score(x_up_down_test,y_up_down_test))

LDA_ovo_u_r = LinearDiscriminantAnalysis()
LDA_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(LDA_ovo_u_r .score(x_right_up_test,y_right_up_test))

LDA_ovo_u_l = LinearDiscriminantAnalysis()
LDA_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(LDA_ovo_u_l.score(x_left_up_test,y_left_up_test))

LDA_ovo_d_r = LinearDiscriminantAnalysis()
LDA_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(LDA_ovo_d_r.score(x_right_down_test,y_right_down_test))

LDA_ovo_d_l = LinearDiscriminantAnalysis()
LDA_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(LDA_ovo_d_l.score(x_left_down_test,y_left_down_test))

LDA_ovo_r_l = LinearDiscriminantAnalysis()
LDA_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(LDA_ovo_r_l.score(x_left_right_test,y_left_right_test))

LDA_ovm_u = LinearDiscriminantAnalysis()
LDA_ovm_u.fit(x_up_train,y_up_train)
print(LDA_ovm_u.score(x_up_test,y_up_test))

LDA_ovm_r = LinearDiscriminantAnalysis()
LDA_ovm_r.fit(x_right_train,y_right_trai76n)
print(LDA_ovm_r.score(x_right_test,y_right_test))

LDA_ovm_d = LinearDiscriminantAnalysis()
LDA_ovm_d.fit(x_down_train,y_down_train)
print(LDA_ovm_d.score(x_down_test,y_down_test))

LDA_ovm_l = LinearDiscriminantAnalysis()
LDA_ovm_l.fit(x_left_train,y_left_train)
print(LDA_ovm_l.score(x_left_test,y_left_test))



Here I am doing the exact same thing except with an SVM classifier. The code has a nearly identical structure and therefore I will not comment it. SVM works by finding the hyperplane that seperates the 2 classes with the largest margin. This means that the distance between the points and seperating boundary is maximised which makes the classification more clear cut. This can be generalised to multiclass classification using OvO classification. I have specified in the initialisation of the model that probability=True which is an instruction to use platt scaling to generate the posterior probabilities and therefore the confidence of the model in all the classes.

In [ ]:
multiclass_SVM = svm.SVC(probability=True)
multiclass_SVM.fit(x_csp_space_train,y_csp_space_train)
print(multiclass_SVM.score(x_csp_space_test,y_csp_space_test))

SVM_binary_search_v_h = svm.SVC(probability=True)
SVM_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
print(SVM_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

SVM_ovo_u_d = svm.SVC(probability=True)
SVM_ovo_u_d.fit(x_up_down_train,y_up_down_train)
print(SVM_ovo_u_d.score(x_up_down_test,y_up_down_test))

SVM_ovo_u_r = svm.SVC(probability=True)
SVM_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(SVM_ovo_u_r .score(x_right_up_test,y_right_up_test))

SVM_ovo_u_l = svm.SVC(probability=True)
SVM_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(SVM_ovo_u_l.score(x_left_up_test,y_left_up_test))

SVM_ovo_d_r = svm.SVC(probability=True)
SVM_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(SVM_ovo_d_r.score(x_right_down_test,y_right_down_test))

SVM_ovo_d_l = svm.SVC(probability=True)
SVM_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(SVM_ovo_d_l.score(x_left_down_test,y_left_down_test))

SVM_ovo_r_l = svm.SVC(probability=True)
SVM_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(SVM_ovo_r_l.score(x_left_right_test,y_left_right_test))

SVM_ovm_u = svm.SVC(probability=True)
SVM_ovm_u.fit(x_up_train,y_up_train)
print(SVM_ovm_u.score(x_up_test,y_up_test))

SVM_ovm_r = svm.SVC(probability=True)
SVM_ovm_r.fit(x_right_train,y_right_train)
print(SVM_ovm_r.score(x_right_test,y_right_test))

SVM_ovm_d = svm.SVC(probability=True)
SVM_ovm_d.fit(x_down_train,y_down_train)
print(SVM_ovm_d.score(x_down_test,y_down_test))

SVM_ovm_l = svm.SVC(probability=True)
SVM_ovm_l.fit(x_left_train,y_left_train)
print(SVM_ovm_l.score(x_left_test,y_left_test))

The exact same thing is done with the Logistic Regresion. LR works by taking a linear combination of all the features, adding a bias and applying a sigmoid function to convert the value into a posterior probability automatically making the output the confidence of the model in one of the 2 classes. This can be generalised to multiclass classification using multinomial LR which essentially repeates the process for all the classes and applies a softmax transform rather than a sigmoid function. The weights and biases are optimised using gradient descent. lbfgs is a variant of this optimisation algorithm that is used on multinomial LR. The details of how it works is not relevant

In [ ]:
multiclass_LR = LogisticRegression(multi_class='multinomial', solver='lbfgs')
multiclass_LR.fit(x_csp_space_train,y_csp_space_train)
print(multiclass_LR.score(x_csp_space_test,y_csp_space_test))

LR_binary_search_v_h = LogisticRegression()
LR_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
print(LR_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

LR_ovo_u_d = LogisticRegression()
LR_ovo_u_d.fit(x_up_down_train,y_up_down_train)
print(LR_ovo_u_d.score(x_up_down_test,y_up_down_test))

LR_ovo_u_r = LogisticRegression()
LR_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(LR_ovo_u_r .score(x_right_up_test,y_right_up_test))

LR_ovo_u_l = LogisticRegression()
LR_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(LR_ovo_u_l.score(x_left_up_test,y_left_up_test))

LR_ovo_d_r = LogisticRegression()
LR_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(LR_ovo_d_r.score(x_right_down_test,y_right_down_test))

LR_ovo_d_l = LogisticRegression()
LR_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(LR_ovo_d_l.score(x_left_down_test,y_left_down_test))

LR_ovo_r_l = LogisticRegression()
LR_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(LR_ovo_r_l.score(x_left_right_test,y_left_right_test))

LR_ovm_u = LogisticRegression()
LR_ovm_u.fit(x_up_train,y_up_train)
print(LR_ovm_u.score(x_up_test,y_up_test))

LR_ovm_r = LogisticRegression()
LR_ovm_r.fit(x_right_train,y_right_train)
print(LR_ovm_r.score(x_right_test,y_right_test))

LR_ovm_d = LogisticRegression()
LR_ovm_d.fit(x_down_train,y_down_train)
print(LR_ovm_d.score(x_down_test,y_down_test))

LR_ovm_l = LogisticRegression()
LR_ovm_l.fit(x_left_train,y_left_train)
print(LR_ovm_l.score(x_left_test,y_left_test))

This is the convolutional neural network. I will not comment every single line as the code for each layer is very repetitive so I will only do the first few.

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
#initialising neural network
model = models.Sequential()
#specifying topology of neural network
#the first layer is 22 2 dimensional convolutions of size (11,1) with stride (1,1)
# the output dimension will be (375-11+1,22-1+1,22)=(365,22,22)
model.add(layers.Conv2D(22, (11,1), (1,1),input_shape=(375,22,1),padding="valid"))
#the next layer is a LeakyReLU activation where the contstant epsilon is 0.01
model.add(layers.LeakyReLU(0.01))
#the next layer is spatial dropout which whill ignore each of the 22 feature maps with probability 0.5
model.add(layers.SpatialDropout2D(.5))
#the next layer is 44 2 dimesnional convolutions of size (1,9) with stride (1,1)
# the output dimension will be (365-1+1,22-8+1,44)=(365,14,44)
model.add(layers.Conv2D(44, (1,9), (1,1),padding="valid"))
#the next layer is a batch normalisation layer
model.add(layers.BatchNormalization())
#the next layer is a LeakyReLU activation where the contstant epsilon is 0.01
model.add(layers.LeakyReLU(0.01))
#the next layer is a max pooling layer
#the output dimension will be (365//2,14,44)=(182,14,44)
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
#the next layer is 88 2 dimesnional convolutions of size (11,1) with stride (1,1)
#output dimension is (182-11+1,14-1+1,88)=(172,14,88)
model.add(layers.Conv2D(88, (11,1), (1,1),padding="valid"))
model.add(layers.LeakyReLU(0.01))
model.add(layers.SpatialDropout2D(.5))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(88, (11,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.SpatialDropout2D(.5))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(176, (2,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(352, (2,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
#This flattens the output of the previous layer into a 1d array that will now be fed into a dense layer
model.add(layers.Flatten())
#a dense layer with 4 outputs
model.add(layers.Dense(4))
#applying softmax transform to get probabilties
model.add(layers.Softmax())
#specifying training mechanism
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
#training neural network
model.fit(np.array(x_train_nn_space),np.array(y_train_nn_space),epochs=500,validation_data=(np.array(x_test_nn_space), np.array(y_test_nn_space)))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 365, 22, 22)       264       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 365, 22, 22)       0         
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 365, 22, 22)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 365, 14, 44)       8756      
_________________________________________________________________
batch_normalization (BatchNo (None, 365, 14, 44)       176       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 365, 14, 44)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 182, 14, 44)       0

The rest of the code is simply applying all the trained models and then taking their outputs and passing them through the relevant functions to obtain the final classification and therefore the overall accuracy.

In [ ]:
LDA_binary_outputs=[]
LDA_ovo_outputs=[]
LDA_ovm_outputs=[]
LDA_multiclass_outputs=[]
SVM_binary_outputs=[]
SVM_ovo_outputs=[]
SVM_ovm_outputs=[]
SVM_multiclass_outputs=[]
LR_binary_outputs=[]
LR_ovo_outputs=[]
LR_ovm_outputs=[]
LR_multiclass_outputs=[]

In [ ]:
for i in x_all_test:
  current=feature_extraction(i,csp_multiclass)
  LDA_multiclass_outputs.append(multiclass_LDA.predict([current]))
  SVM_multiclass_outputs.append(multiclass_SVM.predict([current]))
  LR_multiclass_outputs.append(multiclass_LR.predict([current]))
print(accuracy(LDA_multiclass_outputs,y_all_test))
print(accuracy(SVM_multiclass_outputs,y_all_test))
print(accuracy(LR_multiclass_outputs,y_all_test))

In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_up_right)
  current_2=feature_extraction(i,csp_filters_up_down)
  current_3=feature_extraction(i,csp_filters_up_left)
  current_4=feature_extraction(i,csp_filters_down_right)
  current_5=feature_extraction(i,csp_filters_right_left)
  current_6=feature_extraction(i,csp_filters_down_left)
  output_1=LDA_ovo_u_r.predict([current_1])
  output_2=LDA_ovo_u_d.predict([current_2])
  output_3=LDA_ovo_u_l.predict([current_3])
  output_4=LDA_ovo_d_r.predict([current_4])
  output_5=LDA_ovo_r_l.predict([current_5])
  output_6=LDA_ovo_d_l.predict([current_6])
  LDA_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))
  output_1=SVM_ovo_u_r.predict([current_1])
  output_2=SVM_ovo_u_d.predict([current_2])
  output_3=SVM_ovo_u_l.predict([current_3])
  output_4=SVM_ovo_d_r.predict([current_4])
  output_5=SVM_ovo_r_l.predict([current_5])
  output_6=SVM_ovo_d_l.predict([current_6])
  SVM_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))
  output_1=LR_ovo_u_r.predict([current_1])
  output_2=LR_ovo_u_d.predict([current_2])
  output_3=LR_ovo_u_l.predict([current_3])
  output_4=LR_ovo_d_r.predict([current_4])
  output_5=LR_ovo_r_l.predict([current_5])
  output_6=LR_ovo_d_l.predict([current_6])
  LR_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))

print(accuracy(LDA_ovo_outputs,y_all_test))
print(accuracy(SVM_ovo_outputs,y_all_test))
print(accuracy(LR_ovo_outputs,y_all_test))

In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_v_h)
  current_2=feature_extraction(i,csp_filters_up_down)
  current_3=feature_extraction(i,csp_filters_right_left)
  output_1=LDA_binary_search_v_h.predict([current_1])
  output_2=LDA_ovo_u_d.predict([current_2])
  output_3=LDA_ovo_r_l.predict([current_3])
  LDA_binary_outputs.append(binary_search([output_1,output_2,output_3]))
  output_1=SVM_binary_search_v_h.predict([current_1])
  output_2=SVM_ovo_u_d.predict([current_2])
  output_3=SVM_ovo_r_l.predict([current_3])
  SVM_binary_outputs.append(binary_search([output_1,output_2,output_3]))
  output_1=LR_binary_search_v_h.predict([current_1])
  output_2=LR_ovo_u_d.predict([current_2])
  output_3=LR_ovo_r_l.predict([current_3])
  LR_binary_outputs.append(binary_search([output_1,output_2,output_3]))
print(accuracy(LDA_binary_outputs,y_all_test))
print(accuracy(SVM_binary_outputs,y_all_test))
print(accuracy(LR_binary_outputs,y_all_test))


In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_up)
  current_2=feature_extraction(i,csp_filters_down)
  current_3=feature_extraction(i,csp_filters_left)
  current_4=feature_extraction(i,csp_filters_right)
  output_1=(LDA_ovm_u.predict_proba([current]))[0][0]
  output_2=(LDA_ovm_d.predict_proba([current]))[0][0]
  output_3=(LDA_ovm_l.predict_proba([current]))[0][0]
  output_4=(LDA_ovm_r.predict_proba([current]))[0][0]
  LDA_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
  output_1=(SVM_ovm_u.predict_proba([current]))[0][0]
  output_2=(SVM_ovm_d.predict_proba([current]))[0][0]
  output_3=(SVM_ovm_l.predict_proba([current]))[0][0]
  output_4=(SVM_ovm_r.predict_proba([current]))[0][0]
  SVM_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
  output_1=(LR_ovm_u.predict_proba([current]))[0][0]
  output_2=(LR_ovm_d.predict_proba([current]))[0][0]
  output_3=(LR_ovm_l.predict_proba([current]))[0][0]
  output_4=(LR_ovm_r.predict_proba([current]))[0][0]
  LR_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
print(accuracy(LDA_ovm_outputs,y_all_test))
print(accuracy(SVM_ovm_outputs,y_all_test))
print(accuracy(LR_ovm_outputs,y_all_test))


